## testing/debugging first src funnction created

## 

In [23]:
import os, pathlib, sys
print("CWD:", os.getcwd())
print("src exists?", pathlib.Path("src").exists())
print("__init__.py exists?", pathlib.Path("src/__init__.py").exists())
print("ingest.py exists?", pathlib.Path("src/ingest.py").exists())

CWD: /Users/abdulrahmanaboluhom/Documents/GitHub/Projects/indepth-driving-data-analysis
src exists? True
__init__.py exists? True
ingest.py exists? True


In [24]:
import importlib, types

import src.ingest as ingest
importlib.reload(ingest)     # picks up latest file contents

print("Attributes on ingest:", [a for a in dir(ingest) if not a.startswith("_")])

Attributes on ingest: ['Iterable', 'List', 'Optional', 'Path', 'Union', 'guess_header_row', 'load_journey_event', 'pd']


In [25]:
from src.ingest import load_journey_event

In [26]:
# testing function
# import the loader
from src.ingest import load_journey_event

file = "data/raw/driving_sample.xlsx"
sheet = "Journey_Event_Sample"

df = load_journey_event(
    xlsx_path=file,
    sheet_name=sheet,
    header="auto",                      # or 0 if you want to force it
    usecols="B:BC",                     # skip blank col A
    nrows=400,                          # sample size for speed
    date_cols=["RTC Date Time","GPS Date Time","Event Time Stamp"],
    dayfirst=True
)

df.head(2), df.shape

(  Policy Number  Voucher ID PlateNumber  VIN  Device Serial Number  \
 0   P75235539-1     2307976     SL14CAU  NaN             128491027   
 1   P75235539-1     2307976     SL14CAU  NaN             128491027   
 
               Journey ID Journey Segment Type  Event Type ID  \
 0  230797620241128203458                    D              1   
 1  230797620241128203458                    t              3   
 
                       Event Type       RTC Date Time  ...  \
 0  Departure Event - ignition on 2024-11-28 20:34:58  ...   
 1                Trip Time Event 2024-11-28 20:35:58  ...   
 
   Delta Decelerations 1 Delta Decelerations 2  Delta Decelerations 3  \
 0                     0                     0                      0   
 1                     0                     0                      0   
 
    Delta Decelerations 4  Delta Decelerations 5  Delta Decelerations 6  \
 0                      0                      0                      0   
 1                      0    

# 01_explore — detect header, load 400 rows, parse dates
Goal: robustly load from the original sheet (with legend rows) and confirm key columns/types.

In [1]:
#Environment & CWD (Code)
import os, pathlib, sys, pandas as pd

# Ensure CWD is repo root (parent of notebooks/)
if pathlib.Path().resolve().name == "notebooks":
    os.chdir("..")
print("CWD:", os.getcwd())
print("Python:", sys.version.split()[0], "| pandas:", pd.__version__)

CWD: /Users/abdulrahmanaboluhom/Documents/GitHub/Projects/indepth-driving-data-analysis
Python: 3.9.6 | pandas: 2.3.1


In [2]:
#Data path & sheet list (Code)

from pathlib import Path
DATA = Path("data/raw")
file = DATA / "driving_sample.xlsx"  # <-- change to actual filename
print("File exists:", file.exists())

xls = pd.ExcelFile(file)
xls.sheet_names  # check the exact messy sheet name here

File exists: True


['Journey_Event_Sample']

In [3]:
#Auto detect header row

import numpy as np
import pandas as pd

sheet = "Journey_Event_Sample"   # <-- exact name from the list above

def guess_header_row(xlsx_path, sheet_name, look_rows=40):
    tmp = pd.read_excel(xlsx_path, sheet_name=sheet_name, header=None, nrows=look_rows)
    scores = []
    for i in range(len(tmp)):
        row = tmp.iloc[i]
        nonnull = row.notna().sum()
        texty = sum(isinstance(v, str) for v in row)
        # longer unique strings often indicate headers (vs. repeated numbers/zeros)
        uniq = row.nunique(dropna=True)
        scores.append((i, int(nonnull), int(texty), int(uniq)))
    # sort by non-null, then texty, then unique count
    scores.sort(key=lambda t: (t[1], t[2], t[3]), reverse=True)
    return scores[0][0]  # 0-based index

header_idx = guess_header_row(file, sheet)
print("Guessed header row (0-based):", header_idx, "=> Excel row", header_idx+1)

Guessed header row (0-based): 15 => Excel row 16


In [4]:
peek = pd.read_excel(file, sheet_name=sheet, header=None, skiprows=max(0, header_idx-1), nrows=5)
peek

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,NaN,ALL DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Policy Number,Voucher ID,PlateNumber,VIN,Device Serial Number,Journey ID,Journey Segment Type,Event Type ID,Event Type,...,Delta Decelerations 1,Delta Decelerations 2,Delta Decelerations 3,Delta Decelerations 4,Delta Decelerations 5,Delta Decelerations 6,Delta Decelerations 7,Delta Decelerations 8,Delta Decelerations 9,Delta Decelerations 10
2,NaN,P75235539-1,2307976,SL14CAU,NaN,00128491027,230797620241128203458,D,1,Departure Event - ignition on,...,0,0,0,0,0,0,0,0,0,0
3,NaN,P75235539-1,2307976,SL14CAU,NaN,00128491027,230797620241128203458,t,3,Trip Time Event,...,0,0,0,0,0,0,0,0,0,0
4,NaN,P75235539-1,2307976,SL14CAU,NaN,00128491027,230797620241128203458,t,3,Trip Time Event,...,0,0,0,0,0,0,0,0,0,0


In [5]:
import pandas as pd

# you already have these from earlier cells
# file = DATA / "driving_sample.xlsx"
# sheet = "Journey_Event_Sample"
# header_idx = <the detected 0-based header row index>

df = pd.read_excel(
    file,
    sheet_name=sheet,
    header=header_idx,
    nrows=400,        # your sample size
    usecols="B:BC"    # skip blank/legend col A; adjust end column if needed
)
df.head(2)

,Policy Number,Voucher ID,PlateNumber,VIN,Device Serial Number,Journey ID,Journey Segment Type,Event Type ID,Event Type,RTC Date Time,...,Delta Decelerations 1,Delta Decelerations 2,Delta Decelerations 3,Delta Decelerations 4,Delta Decelerations 5,Delta Decelerations 6,Delta Decelerations 7,Delta Decelerations 8,Delta Decelerations 9,Delta Decelerations 10
0,P75235539-1,2307976,SL14CAU,NaN,128491027,230797620241128203458,D,1,Departure Event - ignition on,28 Nov 2024 20:34:58,...,0,0,0,0,0,0,0,0,0,0
1,P75235539-1,2307976,SL14CAU,NaN,128491027,230797620241128203458,t,3,Trip Time Event,28 Nov 2024 20:35:58,...,0,0,0,0,0,0,0,0,0,0


In [6]:
print("shape:", df.shape)
print("first 12 columns:", df.columns[:12].tolist())

# QUICK: are the three date columns even present?
date_cols = ["RTC Date Time", "GPS Date Time", "Event Time Stamp"]
present = [c for c in date_cols if c in df.columns]
missing = [c for c in date_cols if c not in df.columns]
print("date cols present:", present, "| missing:", missing)

# show dtypes of a few key columns
cols_probe = present + [c for c in ["Event Type","Latitude","Longitude","Horizontal Speed","Road Speed Limit"] if c in df.columns]
df[cols_probe].dtypes

shape: (384, 54)
first 12 columns: ['Policy Number', 'Voucher ID', 'PlateNumber', 'VIN', 'Device Serial Number', 'Journey ID', 'Journey Segment Type', 'Event Type ID', 'Event Type', 'RTC Date Time', 'GPS Date Time', 'Event Time Stamp']
date cols present: ['RTC Date Time', 'GPS Date Time', 'Event Time Stamp'] | missing: []


RTC Date Time        object
GPS Date Time        object
Event Time Stamp     object
Event Type           object
Latitude            float64
Longitude           float64
Horizontal Speed      int64
Road Speed Limit      int64
dtype: object

In [7]:
for col in present:
    df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")

In [8]:
for col in present:
    dtype_ok = str(df[col].dtype).startswith("datetime64")
    nulls = int(df[col].isna().sum())
    print(f"{col}: dtype_ok={dtype_ok}, nulls={nulls} of {len(df)}")

RTC Date Time: dtype_ok=True, nulls=0 of 384
GPS Date Time: dtype_ok=True, nulls=0 of 384
Event Time Stamp: dtype_ok=True, nulls=0 of 384


In [9]:
for col in present:
    bad = df.loc[df[col].isna(), col]
    if len(bad) > 0:
        # show a few original strings (need the original column to compare—if you need this later,
        # we can clone the original text first before to_datetime; for now we just report counts)
        print(f"{col}: {len(bad)} nulls after parsing (inspect upstream if unexpected)")

In [10]:
# Hard guarantees (adjust as you like)
assert all(str(df[c].dtype).startswith("datetime64") for c in present), "Some date columns are not datetime"
print("✅ date dtype assertions passed")

✅ date dtype assertions passed


## ✅ Stopped here
Dates are now parsed + audited. Next step = deeper cleaning & exploring columns.

In [11]:
import pandas as pd

if 'df' in globals():
    print("Reusing df in memory:", df.shape)
else:
    file = "data/raw/driving_sample.xlsx"
    sheet = "Journey_Event_Sample"
    header_idx = 0   # your sample has headers on the first row
    df = pd.read_excel(
        file, sheet_name=sheet, header=header_idx, nrows=400, usecols="B:BC"
    )
    print("Loaded df:", df.shape)

Reusing df in memory: (384, 54)


In [12]:
# column auditing

rows = []
for col in df.columns:
    s = df[col]
    rows.append({
        "column": col,
        "dtype": str(s.dtype),
        "null_pct": round(100 * (1 - s.notna().mean()), 2),
        "n_unique": int(s.nunique(dropna=True)),
    })
audit = pd.DataFrame(rows).sort_values(["dtype","column"]).reset_index(drop=True)
audit.head(15)

,column,dtype,null_pct,n_unique
0,Event Time Stamp,datetime64[ns],0.00,384
1,GPS Date Time,datetime64[ns],0.00,384
2,RTC Date Time,datetime64[ns],0.00,384
3,Latitude,float64,0.00,352
4,Longitude,float64,0.00,359
5,Time Elapsed,float64,3.12,10
6,VIN,float64,100.00,0
7,Accumulated Trip Distance,int64,0.00,192
8,Accumulated Trip Idle Time,int64,0.00,140
9,Accumulated Trip Run Time,int64,0.00,102


In [13]:
keep_cols = [
    # identifiers
    "Policy Number", "Voucher ID", "Journey ID",
    # time
    "Event Time Stamp", "GPS Date Time",
    # location
    "Latitude", "Longitude",
    # motion/context
    "Event Type", "Horizontal Speed", "Road Speed Limit",
    # cumulative metrics
    "Accumulated Trip Distance", "Accumulated Trip Idle Time", "Accumulated Trip Run Time",
]

keep_cols = [c for c in keep_cols if c in df.columns]
df_keep = df[keep_cols].copy()
print("df_keep shape:", df_keep.shape)
df_keep.columns.tolist()

df_keep shape: (384, 13)


['Policy Number',
 'Voucher ID',
 'Journey ID',
 'Event Time Stamp',
 'GPS Date Time',
 'Latitude',
 'Longitude',
 'Event Type',
 'Horizontal Speed',
 'Road Speed Limit',
 'Accumulated Trip Distance',
 'Accumulated Trip Idle Time',
 'Accumulated Trip Run Time']

In [27]:
# calling src.validate
from src.validate import validate_journey_event
validate_journey_event(df_keep)  # raises SchemaError if anything is off
print("✅ Schema validation passed")

✅ Schema validation passed


In [14]:
df_keep.columns

Index(['Policy Number', 'Voucher ID', 'Journey ID', 'Event Time Stamp',
       'GPS Date Time', 'Latitude', 'Longitude', 'Event Type',
       'Horizontal Speed', 'Road Speed Limit', 'Accumulated Trip Distance',
       'Accumulated Trip Idle Time', 'Accumulated Trip Run Time'],
      dtype='object')

In [16]:
from pathlib import Path
Path("data/interim").mkdir(parents=True, exist_ok=True)
df_keep.to_parquet("data/interim/journey_event_sample_keep.parquet", index=False)

In [28]:
probe = ["Event Type","Delta Trip Distance","Direction","Altitude","GPS Accuracy"]
[c for c in probe if c in df.columns], df[probe].head(2)

(['Event Type',
  'Delta Trip Distance',
  'Direction',
  'Altitude',
  'GPS Accuracy'],
                       Event Type  Delta Trip Distance  Direction  Altitude  \
 0  Departure Event - ignition on                    0          0       266   
 1                Trip Time Event                    0        331       244   
 
    GPS Accuracy  
 0             0  
 1            93  )